# Implementation of False Positive Algorithms

In [17]:
import json
import mysql.connector
import sql_commands
import random
random.seed(2311)

In [18]:
mydb, mycursor = sql_commands.connect_to_db("tpch_og")

In [19]:
all_nations = sql_commands.run_query(mycursor, "SELECT N_NAME FROM NATION")
index = random.randint(0, len(all_nations)-1)
nation = all_nations[index][0]
print("Nation chosen:", nation)

Nation chosen: EGYPT


In [20]:
def q1_fps(a,lineitem_cols,query1_cols):
    # Query to get all items from Lineitem table
    lineitem = sql_commands.run_query(mycursor,"SELECT * FROM LINEITEM")
    # Variables needed for comparing lineitem items to query 1 answer
    l_orderkey_index = lineitem_cols.index('L_ORDERKEY')
    o_orderkey_index = query1_cols.index('O_ORDERKEY')
    l_suppkey_index = lineitem_cols.index('L_SUPPKEY')
    s_suppkey_index = query1_cols.index("S_SUPPKEY")
    l_commitdate_index = lineitem_cols.index("L_COMMITDATE")
    l_receiptdate_index = lineitem_cols.index('L_RECEIPTDATE')
    count = 0
    for t in lineitem:
        if t[l_orderkey_index] == a[o_orderkey_index]:
            x = t[l_suppkey_index]
            if x is not None and x == a[s_suppkey_index]:
                continue
            
            d1 = t[l_commitdate_index]
            d2 = t[l_receiptdate_index]
            if d1 is None or d2 is None or d2 > d1:
                count += 1
                
    return count

In [21]:
# list of tuples
# (suppkey, orderkey)
mydb, mycursor = sql_commands.connect_to_db("tpch_10pct")
query1_out = sql_commands.run_query1(mycursor,nation)
#TODO: implement fp alg 1 and calculate #fp for query1
lineitem_attributes = sql_commands.get_attribute_names(mycursor,"LINEITEM")
query1_attributes = ['S_SUPPKEY', 'O_ORDERKEY']
print(lineitem_attributes)
print(query1_attributes)
fp_count = q1_fps(query1_out, lineitem_attributes, query1_attributes)
print(f"{fp_count} false positives found in Query 1")

['L_ORDERKEY', 'L_PARTKEY', 'L_SUPPKEY', 'L_LINENUMBER', 'L_QUANTITY', 'L_EXTENDEDPRICE', 'L_DISCOUNT', 'L_TAX', 'L_RETURNFLAG', 'L_LINESTATUS', 'L_SHIPDATE', 'L_COMMITDATE', 'L_RECEIPTDATE', 'L_SHIPINSTRUCT', 'L_SHIPMODE', 'L_COMMENT']
['S_SUPPKEY', 'O_ORDERKEY']
0 false positives found in Query 1


In [22]:
sql_commands.close_db(mydb, mycursor)